In [21]:
import cv2
import sys


In [22]:
def get_data(input_path):
    """Parse the data from annotation file

	Args:
		input_path: annotation file path

	Returns:
		all_data: list(filepath, width, height, list(bboxes))
		classes_count: dict{key:class_name, value:count_num}
			e.g. {'Car': 2383, 'Mobile phone': 1108, 'Person': 3745}
		class_mapping: dict{key:class_name, value: idx}
			e.g. {'Car': 0, 'Mobile phone': 1, 'Person': 2}
	"""
    found_bg = False
    all_imgs = {}

    classes_count = {}

    class_mapping = {}

    i = 1

    with open(input_path, 'r') as f:

        print('Parsing annotation files')

        for line in f:

            # Print process
            sys.stdout.write('\r' + 'idx=' + str(i))
            i += 1

            line_split = line.strip().split(',')

            # Make sure the info saved in annotation file matching the format (path_filename, x1, y1, x2, y2, class_name)
            # Note:
            #	One path_filename might has several classes (class_name)
            #	x1, y1, x2, y2 are the pixel value of the origial image, not the ratio value
            #	(x1, y1) top left coordinates; (x2, y2) bottom right coordinates
            #   x1,y1-------------------
            #	|						|
            #	|						|
            #	|						|
            #	|						|
            #	---------------------x2,y2

            (filename, class_name, x1, y1, x2, y2) = line_split

            if class_name not in classes_count:
                classes_count[class_name] = 1
            else:
                classes_count[class_name] += 1

            if class_name not in class_mapping:
                class_mapping[class_name] = len(class_mapping)

            if filename not in all_imgs:
                all_imgs[filename] = {}

                img = cv2.imread('Dataset/' + filename)
                (rows, cols) = img.shape[:2]
                all_imgs[filename]['filepath'] = filename
                all_imgs[filename]['image'] = img
                all_imgs[filename]['width'] = cols
                all_imgs[filename]['height'] = rows
                all_imgs[filename]['bboxes'] = []

            all_imgs[filename]['bboxes'].append({
                                        'class_id': class_mapping[class_name],
                                        'x_center': (int(x1) + int(x2)) / 2.0 / cols,
                                        'y_center': (int(y1) + int(y2)) / 2.0 / rows,
                                        'width': (int(x2) - int(x1)) / cols,
                                        'height': (int(y2) - int(y1)) / rows})

        all_data = []
        for key in all_imgs:
            all_data.append(all_imgs[key])

        return all_data, classes_count, class_mapping


In [23]:
trainImages, classesCount, classMapping= get_data('data.csv')

Parsing annotation files
idx=4888

In [24]:
import yaml

In [44]:
from pathlib import Path

def splitData(numericalDataset,trainName:str,testName:str,trainSize:float=0.8):
    splitIndex = int(trainSize * len(numericalDataset))
    train:list=[]
    test:list=[]
    for i, file in enumerate(numericalDataset):
        if i < splitIndex:
            train.append(file)
            filename= Path(trainName) / Path(file['filepath'])
            cv2.imwrite(filename,file['image'])
        else:
            test.append(file)
            filename= Path(testName) / Path(file['filepath'])
            cv2.imwrite(filename,file['image'])

    return train, test

In [45]:
import os

def create_yolo_labels(dataset, class_mapping, output_dir='labels'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for item in dataset:
        filename = item['filepath']
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(output_dir, label_filename)
        
        with open(label_path, 'w') as f:
            for bbox in item['bboxes']:
                line = f"{bbox['class_id']} {bbox['x_center']} {bbox['y_center']} {bbox['width']} {bbox['height']}\n"
                f.write(line)

In [46]:
def create_yolo_yaml(data_paths, class_mapping, output_path='data.yaml'):
    sorted_classes = [class_name for class_name, idx in 
                     sorted(class_mapping.items(), key=lambda x: x[1])]
    
    yaml_data = {
        'path': '.', 
        'train': 'train',
        'val': 'val',
        'test': 'test',
        'nc': len(sorted_classes), 
        'names': sorted_classes  
    }
    
    with open(output_path, 'w') as f:
        yaml.dump(yaml_data, f, default_flow_style=False, sort_keys=False)

    return yaml_data


In [47]:
dataCombined:dict={}
dataCombined['train'], dataCombined['test'] = splitData(trainImages,trainName='train/images',testName='test/images')
dummyTrain,dataCombined['val']=splitData(dataCombined['train'],trainName='train/images',testName='validation/images')
datasetYAMLformat = create_yolo_yaml(dataCombined,classMapping)

In [34]:
create_yolo_labels(dataCombined['train'],classMapping,"train/labels")
create_yolo_labels(dataCombined['test'],classMapping,"test/labels")
create_yolo_labels(dataCombined['val'],classMapping,"val/labels")

In [30]:
from ultralytics import YOLO


In [33]:
firstModel = YOLO("yolo11s.pt")

# firstModel.train(
#     data="data.yaml",  
#     epochs=50,                 
#     imgsz=640,                 
#     batch=16,                  
#     name="asl_yolo11"          
# )